In [174]:
import pandas as pd 
import numpy as np 
import statsmodels.api as sm

In [191]:
ev = pd.read_csv('ev_county_clean.csv')
income = pd.read_csv('wa_median_income.csv')
sex = pd.read_csv('Population by gender.csv')

In [176]:
# added ev total to model df
ev_model = ev[ev['Date'] == 'December 31 2021']
ev_model = ev_model.drop(columns=['Date','Battery Electric Vehicles (BEVs)','Plug-In Hybrid Electric Vehicles (PHEVs)','Non-Electric Vehicle Total','Total Vehicles'])
ev_model = ev_model.rename(columns={'Electric Vehicle (EV) Total': 'EV Total'})

In [177]:
# merged income df, cleaned df
ev_model = ev_model.merge(income, on='County')
ev_model = ev_model.drop(columns=['Unnamed: 0','2022'])
ev_model = ev_model.rename(columns={'2021': 'Median Income'})

# changed median income from str to int
ev_model['Median Income'] = ev_model['Median Income'].str.replace(',', '')
ev_model['Median Income'] = ev_model['Median Income'].astype(int)

In [178]:
# dropped all columns except for chosen
sex = sex.loc[:, sex.columns.intersection(['Location Name','Indicator Name','2021'])]
sex = sex.dropna()

# removed total state info, formated county 
sex = sex[sex['Location Name'].str.contains('County')]
sex['Location Name'] = sex['Location Name'].str.replace(' County', '')

# changing population number from float to int
sex['2021'] = sex['2021'].astype(int)

In [179]:
# merged male pop. by county to ev model df
sex_m = sex[sex['Indicator Name'] == 'Male']
sex_m = sex_m.drop(columns='Indicator Name')
sex_m = sex_m.rename(columns={'Location Name': 'County','2021': 'Male Pop.'})
ev_model = ev_model.merge(sex_m, on='County')

In [181]:
# merged female pop. by county to ev model df
sex_f = sex[sex['Indicator Name'] == 'Female']
sex_f = sex_f.drop(columns='Indicator Name')
sex_f = sex_f.rename(columns={'Location Name': 'County','2021': 'Female Pop.'})
ev_model = ev_model.merge(sex_f, on='County')

In [182]:
# added total pop. as another variable
ev_model['Pop.'] = ev_model['Male Pop.'] + ev_model['Female Pop.']

In [183]:
# changed male, female pop to ratios
ev_model['Male Pop.'] = ev_model['Male Pop.'] / ev_model['Pop.']
ev_model['Female Pop.'] = ev_model['Female Pop.'] / ev_model['Pop.']
ev_model = ev_model.rename(columns={'Male Pop.': 'Male Ratio', 'Female Pop.': 'Female Ratio'})

In [184]:
X = ev_model.drop(['County', 'EV Total'], axis=1)
y = ev_model['EV Total']

In [185]:
mn = sm.OLS(y,sm.add_constant(X))

In [186]:
model = mn.fit()
print_model = model.summary()
print(print_model)

                            OLS Regression Results                            
Dep. Variable:               EV Total   R-squared:                       0.901
Model:                            OLS   Adj. R-squared:                  0.893
Method:                 Least Squares   F-statistic:                     106.5
Date:                Thu, 02 Nov 2023   Prob (F-statistic):           1.16e-17
Time:                        14:57:18   Log-Likelihood:                -357.16
No. Observations:                  39   AIC:                             722.3
Df Residuals:                      35   BIC:                             729.0
Df Model:                           3                                         
Covariance Type:            nonrobust                                         
                    coef    std err          t      P>|t|      [0.025      0.975]
---------------------------------------------------------------------------------
const          1031.3552   2097.250      0.492